In [ ]:
import os
import json
import warnings
warnings.filterwarnings("ignore")

import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, Point, LineString

ox.settings.log_console = False
ox.settings.use_cache = True

DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)

print("osmnx:", ox.__version__)

osmnx: 2.0.6


In [2]:
PLACE_NAME = "Tsinghua University, Beijing, China"

campus_gdf = ox.geocoder.geocode_to_gdf(PLACE_NAME)
campus_gdf_4326 = campus_gdf.to_crs(4326)
campus_gdf_3857 = campus_gdf.to_crs(3857)

campus_gdf_4326.to_file(os.path.join(DATA_DIR, "campus_boundary.geojson"), driver="GeoJSON")

campus_poly_3857 = campus_gdf_3857.geometry.iloc[0]
campus_poly_4326 = campus_gdf_4326.geometry.iloc[0]
print("Boundary area (km^2):", campus_poly_3857.area / 1e6)

campus_gdf_4326.head()

Boundary area (km^2): 5.127747316965771


,geometry,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((116.31 39.998, 116.31 39.998, 116.31...",116.309263,39.991334,116.330816,40.011946,202857946,relation,7469216,40.002291,116.320963,amenity,university,30,0.595353,amenity,Tsinghua University,"Tsinghua University, 30, Shuangqing Road, 东升镇,..."


In [3]:
def to_wgs84(geom_3857):
    return gpd.GeoSeries([geom_3857], crs=3857).to_crs(4326).iloc[0]


def download_pois_by_tags(polygon_3857, tags, buffer_m=0.0):
    poly = polygon_3857
    if buffer_m and buffer_m != 0:
        poly = gpd.GeoSeries([polygon_3857], crs=3857).buffer(buffer_m).iloc[0]
    poly_wgs84 = to_wgs84(poly)
    gdf = ox.features_from_polygon(poly_wgs84, tags)
    if gdf.empty:
        return gpd.GeoDataFrame(columns=["geometry"], geometry="geometry", crs=4326)
    return gdf


def save_geojson(gdf, name):
    if gdf is None or gdf.empty:
        print(f"{name}: empty")
        return
    out = os.path.join(DATA_DIR, f"{name}.geojson")
    gdf.to_file(out, driver="GeoJSON")
    print(f"saved {name} -> {out}")

In [4]:
import folium
from folium.plugins import Draw

m = folium.Map(
    location=[campus_gdf_4326.centroid.y.iloc[0], campus_gdf_4326.centroid.x.iloc[0]],
    zoom_start=15,
    tiles="cartodbpositron"
)

folium.GeoJson(
    campus_gdf_4326,
    name="boundary",
    style_function=lambda x: {"color": "#333", "weight": 2, "fill": False}
).add_to(m)

draw = Draw(
    export=True, 
    filename="gates_manual.geojson", 
    draw_options={
        "polyline": False,
        "polygon": False,
        "circle": False,
        "rectangle": False,
        "circlemarker": False,
        "marker": True 
    },
    edit_options={"edit": True}
)
draw.add_to(m)

folium.LayerControl().add_to(m)

m


In [5]:
# 建筑
buildings = ox.features_from_polygon(campus_poly_4326, {"building": True})

# 道路
roads = ox.features_from_polygon(campus_poly_4326, {"highway": True})

# 宿舍识别：包含"号楼"的为宿舍
dorms = gpd.GeoDataFrame(columns=["geometry"], geometry="geometry", crs=buildings.crs)
if "name" in buildings.columns:
    dorm_mask = buildings["name"].fillna("").str.contains("号楼", regex=True)
    dorms = buildings[dorm_mask].copy()
    print(f"识别到宿舍: {len(dorms)} 栋")

# 食堂识别：包含"园"字的为食堂
canteens = gpd.GeoDataFrame(columns=["geometry"], geometry="geometry", crs=buildings.crs)
if "name" in buildings.columns:
    canteen_mask = buildings["name"].fillna("").str.contains("园", regex=True)
    canteens = buildings[canteen_mask].copy()
    print(f"识别到食堂: {len(canteens)} 个")

# 校门
gates = gpd.read_file("data/gates_manual.geojson")
print(f"识别到校门: {len(gates)} 个")

# 提取运动场/球场（leisure / sport）
sports = ox.features_from_polygon(campus_poly_4326, {
    "leisure": ["pitch", "track", "stadium"],
    "sport": True
})

if "name" in sports.columns:
    sports_named = sports[sports["name"].fillna("").str.contains("操场|球场", regex=True)]
    print(f"识别到运动场（带名称）: {len(sports_named)} 个")
else:
    sports_named = gpd.GeoDataFrame(columns=["geometry"], geometry="geometry", crs=sports.crs)

print(f"识别到运动场（总数）: {len(sports)} 个")

# 保存
for name, gdf in {
    "buildings": buildings,
    "roads": roads,
    "canteens": canteens,
    "dorms": dorms,
    "gates": gates,
    "sports": sports,
}.items():
    save_geojson(gdf.to_crs(4326) if not gdf.empty else gdf, name)

len(buildings), len(roads), len(canteens), len(dorms), len(gates)

识别到宿舍: 75 栋
识别到食堂: 18 个
识别到校门: 8 个
识别到运动场（带名称）: 20 个
识别到运动场（总数）: 59 个
saved buildings -> data/buildings.geojson
saved roads -> data/roads.geojson
saved canteens -> data/canteens.geojson
saved dorms -> data/dorms.geojson
saved gates -> data/gates.geojson
saved sports -> data/sports.geojson


(626, 931, 18, 75, 8)

In [6]:
import folium

m = folium.Map(
    location=[campus_gdf_4326.centroid.y.iloc[0], campus_gdf_4326.centroid.x.iloc[0]],
    zoom_start=15,
    tiles="cartodbpositron"
)

folium.GeoJson(
    campus_gdf_4326,
    name="boundary",
    style_function=lambda x: {"color": "#333", "weight": 2, "fill": False}
).add_to(m)

layers = [
    ("buildings", "#8888ff"),
    ("roads", "#aaaaaa"),
    ("canteens", "#ff7f0e"),
    ("dorms", "#2ca02c"),
    ("gates", "#d62728"),
    ("sports", "#9467bd"),  
]

for name, color in layers:
    path = os.path.join(DATA_DIR, f"{name}.geojson")
    if os.path.exists(path):
        try:
            g = gpd.read_file(path)
            if not g.empty:
                folium.GeoJson(
                    g,
                    name=name,
                    style_function=lambda x, c=color: {"color": c, "weight": 1}
                ).add_to(m)
        except Exception as e:
            print(f"skip {name}: {e}")

folium.LayerControl().add_to(m)

m


skip roads: Object of type Timestamp is not JSON serializable
